# Predicting FWI using deepFWI Pre-trained models

### Introduction
The Fire Weather Index (FWI) is a meteorologically based index used worldwide to estimate fire danger. It consists of different components that account for the effects of fuel moisture and wind on fire behaviour and spread. The higher the FWI, the more favorable the meteorological conditions to trigger a wildfire. This indicator can help shape long-term tourist strategy and to plan future investments under a changing climate.

The [`esowc/wildfire-forecasting`](https://github.com/esowc/wildfire-forecasting) project intends to reproduce the Fire Forecasting capabilities of GEFF using Deep Learning and develop further improvements in accuracy, geography and time scale through inclusion of additional variables or optimisation of model architecture & hyperparameters. 

In this notebook we demonstrate the use of pre-trained Deep Learning models to perform FWI inference. The Deep Learning Model has been trained on 1 year of data and takes as input the four weather forcings used in the numerical calculation of FWI:
```
* t2     -    Temperature at 2m (K)
* tp     -    Total Precipitation accumulated over the previous 24 hours (mm)
* rh     -    Relative Humidity (%)
* wspeed -    Windspeed (m/s)
```

These input weather forcings are supplied for 7 days and the model produces FWI forecast for 21 days. There is 1 day of overlap between the 4 weather forcing variable inputs and the output FWI forecast as indicated below:
```
                                       ____________
Weather Forcings (t-6, t-5, ..., t) -> | deepFWI  |  -> deepFWI-Forecast (t, t+1, t+2, ..., t+20)
                                       ------------
```

Along with FWI inference using deepGEFF pre-trained models, we also benchmark these results against the baseline `FWI-Forecast` which are numerically calculated FWI predictions from weather forecast.

To run this notebook, make sure you have the `wildfire-dl` `conda` environment activated. Instructions for setting up the environment are available in the project [`README`](https://github.com/esowc/wildfire-forecasting/blob/master/README.md).

In [1]:
import os
os.chdir('..')

### Ensure access to the data
The `/data/data-7-21` contains the following data - 
* `fwi-forcings/`: Weather Forcings for 20xx-xx-xx to 20xx-xx-xx --> 7 days --> 7 files 
* `fwi-reanalysis/`: ERA Reanalysis of FWI for 20xx-xx-xx to 20xx-xx-xx --> 21 days --> 21 files


If you wish to explore these datasets further, have a look at the EDA notebooks in `data/EDA/` in the repository.

### Running the inference and the benchmark

For performing inference and then benchmarking the results, we use the `run(**kwargs)` function from [`src/test.py`](https://github.com/esowc/wildfire-forecasting/blob/master/src/test.py) which takes as argument a dict of configuration parameters and boolean for `benchmark`. You can check out more details about this function in [`test.py#L266`](https://github.com/esowc/wildfire-forecasting/blob/master/src/test.py#L266)

In [4]:
from src.test import run

Providing the configuration options,

```
* in_days        -    no of input days of weather forcings
* out_days       -    no of output days of fwi prediction
* checkpoint_file-    path to pretrained checkpoint file
* reanalysis_dir -    path to fwi-reanalysis data
* forcings_dir   -    path to fwi-forcings data
```

In [5]:
config = dict(
    in_days=7,
    out_days=21,
    checkpoint_file="src/model/checkpoints/unet_tapered-fwi_reanalysis-7-21--10/15-18:37unet_tapered-fwi_reanalysis-7-21--10/15-18:37_ckpt_epoch_29.ckpt",
    reanalysis_dir="/home/esowc/data/ext/fwi-reanalysis",
    forcings_dir="/home/esowc/data/ext/fwi-forcings",
    forecast_dir = ""
)

### Results

We demonstrate below the metrics for deepFWI predicted values and benchmark them against the numerical fwi-forecast prediction values. Accuracy, Mean Squared Error (MSE) and Mean Absolute Error (MAE), being the best evaluation metrics for comparing regression models, these are presented for 1 FWI prediction (for 10 days) in a global setting. The metrics are calculated against fwi-reanalysis (considered the ground truth). The grouped bar plots indicate the metrics for deepFWI while the line plots are used for fwi-forecast. The FWI values are binned as per the EFFIS categorisation:

```
* [0, 5.2]     - Very Low
* (5.2, 11.2]  - Low
* (11.2, 21.3] - Moderate
* (21.3, 38.0] - High
* (38.0, 50.0] - Very High
* (50.0, Inf)  - Extreme
```

As seen, the deepFWI predictions are consistent with FWI-Forecast and even offer better accuracy for longer term predictions in certain FWI categories. 

In [6]:
run(**config, benchmark=False);

TypeError: 'NoneType' object is not subscriptable